# 2.3 自动化评测答疑机器人的表现

## 🚄 前言

新人答疑机器人在实际使用中可能会有一些问题。例如，当新人提问“如何请假”时，机器人可能给出通用的回答，而不是基于制度文件内容进行回答。

和常规的软件开发需要测试一样，你也应该在你的答疑机器人项目中建立一套评测体系，确保在类似的问题都能快速定位原因，并且在每次针对一个问题优化后，能对一批问题进行测试，确保此次优化的对答疑机器人的整体效果是正向的。


## 🍁 课程目标
学完本课程后，你将能够了解：
- 如何自动化大模型应用评测。
- 如何通过 Ragas 对 RAG 应用进行评测。

## 📖 课程目录
在本章节课程中，我们首先会通过一个具体的问题来了解当前 RAG 应用的一些问题，然后尝试通过自己实现一个简单的自动化测试，来发现该问题。最后我们会学习如何使用比较成熟的 RAG 应用测试框架 Ragas 来评估 RAG 应用的效果

- 1.&nbsp;评估 RAG 应用表现
    - 1.1 答疑机器人存在的问题
    - 1.2 查看 RAG 检索结果排查问题
    - 1.3 尝试建立自动化测试机制 
  
- 2.&nbsp;使用 Ragas 来评估应用表现
     - 2.1 评估 RAG 应用回答质量
       - 2.1.1 快速上手
       - 2.1.2 了解 answer correctness 计算过程
     - 2.2 评估检索召回效果
       - 2.2.1 快速上手
       - 2.2.2 了解 context recall 和 context precision 计算过程

## 1. 评估 RAG 应用表现

### 1.1 答疑机器人存在的问题

在前面的章节中，你完成了一个答疑机器人开发，但你发现它目前在员工查询类的问题上表现不佳。

比如，张伟是员工信息表里的第一个员工，但是在你的答疑机器人中却无法回答「张伟是哪个部门的」。

<img src="https://img.alicdn.com/imgextra/i1/O1CN01MsuZGI1E0rrkVNNnO_!!6000000000290-0-tps-1626-278.jpg" width="600">

In [4]:
import os
from config.load_key import load_key
load_key()
print(f'''你配置的 API Key 是：{os.environ["DASHSCOPE_API_KEY"][:5]+"*"*5}''')

你配置的 API Key 是：sk-1a*****


In [6]:
from chatbot import rag
query_engine = rag.create_query_engine(rag.load_index())
print('提问：张伟是哪个部门的')
response = query_engine.query('张伟是哪个部门的')
print('回答：', end='')
response.print_response_stream()

提问：张伟是哪个部门的
回答：根据提供的信息，没有提到张伟所在的部门。如果您能提供更多关于张伟的信息，我可能能够帮助您找到答案。

### 1.2. 查看 RAG 检索结果排查问题
为了解决这个问题，你需要确认在你的答疑机器人回答问题前，召回的参考资料里是否有张伟的相关资料。

通过下面的代码，可以获取到答疑机器人回答这次问题时检索到的参考信息。

In [7]:
contexts = [node.get_content() for node in response.source_nodes]
contexts

['核，提供⾏政管理与协调⽀持，优化⾏政⼯作流程。 \n⾏政部 秦⻜ 蔡静 G705 034 ⾏政 ⾏政专员 13800000034 qinf@educompany.com 维护公司档案与信息系统，负责公司通知及公告的发布，',
 '⽀持。 \n绩效管理部 韩杉 李⻜ I902 041 ⼈⼒资源 绩效专员 13800000041 hanshan@educompany.com 建⽴并维护员⼯绩效档案，定期组织绩效评价会议，协调各部⻔反馈，制定考核流程与标准，确保绩效']

由此可见，这一问题是检索效果不佳造成的。

> 这一章节将专注于建立自动化测试，本身这一检索效果问题将会在后续章节学习中解决。

### 1.3. 尝试建立自动化测试机制

尽管你总是能想办法定位到问题，但是如果每次都要这样去确认是检索出错、还是检索正确但模型生成答案出错，会非常耗时。你应该建立一个测试机制，能够自动地对你准备的一批问题进行测试。


在前面的学习中，我们已经知道大模型可以用来回答问题、检查错误。同样的，大模型也可以用于检测我们问答机器人的问题是否准确回答了问题。

In [8]:
from chatbot import llm

def test_answer(question, answer):
    prompt = ("你是一个测试人员。\n"
        "你需要检测下面的这段回答是否有效回答了用户的问题。\n"
        "回复只能是：有效回答 或者 无效回答。请勿给出其他信息。\n"
        "------"
        f"回答是 {answer}"
        "------"
        "问题是： {question}"
    )
    return llm.invoke(prompt)

test_answer("张伟是哪个部门的", "根据提供的信息，没有提到张伟所在的部门。如果您能提供更多关于张伟的信息，我可能能够帮助您找到答案。")

'无效回答'

In [9]:
def test_contexts(question, contexts):
    prompt = ("你是一个测试人员。\n"
        "你需要检测下面的这些参考资料是否能对回答问题有帮助。\n"
        "回复只能是：参考信息有用 或者 参考信息无用。请勿给出其他信息。\n"
        "------"
        f"参考资料是 {contexts}"
        "------"
        "问题是： {question}"
    )
    return llm.invoke(prompt)
test_contexts("张伟是哪个部门的", "核，提供⾏政管理与协调⽀持，优化⾏政⼯作流程。 \n⾏政部 秦⻜ 蔡静 G705 034 ⾏政 ⾏政专员 13800000034 qinf@educompany.com 维护公司档案与信息系统，负责公司通知及公告的发布，\n\n⽀持。 \n绩效管理部 韩杉 李⻜ I902 041 ⼈⼒资源 绩效专员 13800000041 hanshan@educompany.com 建⽴并维护员⼯绩效档案，定期组织绩效评价会议，协调各部⻔反馈，制定考核流程与标准，确保绩效")

'参考信息无用'

有了上面的两个方法，你已经完成了一个大模型测试工程的雏形。但截至目前的实现还并不完善，比如：

- 因为大模型有时候会捏造事实（幻觉），其给出的答案看起来就像是真的一样，对于这种情况 `test_answer` 方法并不能很好的检测出来。
- 检索召回的参考信息中相关的信息占比越多越好（信噪比），但目前我们的测试方法还比较简单，没有考虑这些。

你可以考虑使用一些成熟的测试框架来进一步完善你的测试工程，比如 [Ragas](https://docs.ragas.io/en/stable)，它是一个专门设计用于评估 RAG 应用表现的测试框架。


## 2. 使用 Ragas 来评估应用表现

Ragas 默认提供了多项指标，可以用来全链路评测应用的问答质量。比如：

- 整体回答质量的评估：
  - Answer Correctness，用于评估 RAG 应用生成答案的准确度。
- 生成环节的评估：
  - Answer Relevancy，用于评估 RAG 应用生成的答案是否与问题相关。
  - Faithfulness，用于评估 RAG 应用生成的答案和检索到的参考资料的事实一致性。
- 召回阶段的评估：
  - Context Precision，用于评估 contexts 中与准确答案相关的条目是否排名靠前、占比高（信噪比）。
  - Context Recall，用于评估有多少相关参考资料被检索到，越高的得分意味着更少的相关参考资料被遗漏。

<img src="https://img.alicdn.com/imgextra/i4/O1CN01b2lVQp21JZCJy6Nfe_!!6000000006964-0-tps-739-420.jpg" width="500">

### 2.1 评估 RAG 应用回答质量

#### 2.1.1 快速上手


在评估 RAG 应用整体回答质量时，使用 Ragas 的 Answer Correctness 是一个很好的指标。为了计算这个指标，你需要准备三种数据：

- **question**，输入给 RAG 应用的问题
- **answer**，RAG 应用给出的答案
- **ground_truth**，你预先知道的正确的答案

这里我们针对问题「*张伟是哪个部门的*」准备了三组 RAG 应用回答：

- **回答不知道**：根据提供的信息，没有提到张伟所在的部门。如果您能提供更多关于张伟的信息，我可能能够帮助您找到答案。
- **捏造答案（幻觉）**：张伟是人事部门的。
- **回答正确**：张伟是教研部的。

然后我们就可以运行下面的代码，来计算回答准确度（即 Answer Correctness）的得分。

In [10]:
from langchain_community.llms.tongyi import Tongyi
from langchain_community.embeddings import DashScopeEmbeddings
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness

data_samples = {
    'question': [
        '张伟是哪个部门的？',
        '张伟是哪个部门的？',
        '张伟是哪个部门的？'
    ],
    'answer': [
        '根据提供的信息，没有提到张伟所在的部门。如果您能提供更多关于张伟的信息，我可能能够帮助您找到答案。',
        '张伟是人事部门的',
        '张伟是教研部的'
    ],
    'ground_truth':[
        '张伟是教研部的成员',
        '张伟是教研部的成员',
        '张伟是教研部的成员'
    ]
}

dataset = Dataset.from_dict(data_samples)
score = evaluate(
    dataset = dataset,
    metrics=[answer_correctness],
    llm=Tongyi(model_name="qwen-plus"),
    embeddings=DashScopeEmbeddings(model="text-embedding-v3")
)
score.to_pandas()

/Users/liuyang/Documents/work/repository/doc/develop/aliyun_acp_learning/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Evaluating: 100%|██████████| 3/3 [00:20<00:00,  6.87s/it]


,question,answer,ground_truth,answer_correctness
0,张伟是哪个部门的？,根据提供的信息，没有提到张伟所在的部门。如果您能提供更多关于张伟的信息，我可能能够帮助您找到答案。,张伟是教研部的成员,0.175227
1,张伟是哪个部门的？,张伟是人事部门的,张伟是教研部的成员,0.193980
2,张伟是哪个部门的？,张伟是教研部的,张伟是教研部的成员,0.994619


可以看到，Ragas 的 Answer Correctness 指标准确的反映出了三种回答的表现，得分越高，越符合事实。

#### 2.1.2 了解 answer correctness 的计算过程

Answer correctness 的得分由生成答案（answer）和正确答案（ground truth）的**语义相似度**和**事实准确度**计算得出。

##### 语义相似度 
语义相似度是通过文本向量模型（Text Embedding Model）计算 answer 和 ground truth 的文本向量，然后计算两个文本向量的余弦相似度。

##### 事实准确度
事实准确度的计算过程稍微复杂一些：

1.  通过大模型将answer、ground_truth分别生成各自的观点列表。比如：  
    - **生成 answer 的观点列表**: 张伟是教研部负责大模型课程的同事。 ---> ["*张伟是教研部的*", "*张伟负责大模型课程*"]  
    - **生成 ground_truth 的观点列表**：张伟是教研部负责大数据方向的同事。---> ["*张伟是教研部的*", "*张伟负责大数据方向*"]
    
2.  遍历answer与ground_truth列表，并初始化三个列表，TP、FP与FN。 
    - 对于由**answer**生成的观点：
      - 如果该观点与ground_truth的观点相匹配，则将该观点添加到TP列表中。比如：「*张伟是教研部的*」。
      - 如果该观点在 ground_truth 的观点列表中找不到依据，则将该观点添加到FP列表中。比如：「*张伟负责大模型课程*」。
    - 对于**ground_truth**生成的观点：
      - 如果该观点在 answer 的观点列表中找不到依据，则将该陈述添加到FN列表中。比如：「*张伟负责大数据方向*」。       
    <!-- ![image](https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/ybEnBBXZ6LoPnP13/img/86c4839b-bd48-4faf-9842-c42599181339.png) -->
    
3.  统计TP、FP与FN列表的元素个数，并按照以下方式计算f1 score分数：    

```shell
f1 score = tp / (tp + 0.5 * (fp + fn)) if tp > 0 else 0
```

以上文为例：f1 score = 1/(1+0.5\*(1+1)) = 0.5

##### 分数汇总
得到语义相似度和事实准确度的分数后，对两者加权求和，即可得到最终的 Answer Correctness 的分数。
```
Answer Correctness 的得分 = 0.25 * 语义相似度得分 + 0.75 * 事实准确度得分
```


### 2.2 评估检索召回效果

#### 2.2.1 快速上手

Ragas 中的 context precision 和 context recall 指标可以用于评估 RAG 应用中的检索的召回效果。

- Context precision 会评估检索召回的参考信息（contexts）中与准确答案相关的条目是否排名靠前、占比高（信噪比），**侧重相关性**。
- Context recall 则会评估 contexts 与 ground_truth 的事实一致性程度，**侧重事实准确度**。

实际应用时，可以将两者结合使用。

为了计算这些指标，你需要准备的数据集应该包括以下信息：

- **question**，输入给 RAG 应用的问题。
- **contexts**，检索召回的惨考信息。
- **ground_truth**，你预先知道的正确的答案。

我们可以继续使用「*张伟是哪个部门的*」这个问题，然后准备三组数据。你可以运行下面的代码，来同时计算 context precision 和 context recall 的得分。

In [11]:
from langchain_community.llms.tongyi import Tongyi
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import context_recall,context_precision

data_samples = {
    'question': [
        '张伟是哪个部门的？',
        '张伟是哪个部门的？',
        '张伟是哪个部门的？'
    ],
    'answer': [
        '根据提供的信息，没有提到张伟所在的部门。如果您能提供更多关于张伟的信息，我可能能够帮助您找到答案。',
        '张伟是人事部门的',
        '张伟是教研部的'
    ],
    'ground_truth':[
        '张伟是教研部的成员',
        '张伟是教研部的成员',
        '张伟是教研部的成员'
    ],
    'contexts' : [
        ['提供⾏政管理与协调⽀持，优化⾏政⼯作流程。 ', '绩效管理部 韩杉 李⻜ I902 041 ⼈⼒资源'],
        ['李凯 教研部主任 ', '牛顿发现了万有引力'],
        ['牛顿发现了万有引力', '张伟 教研部工程师，他最近在负责课程研发'],
    ],
}

dataset = Dataset.from_dict(data_samples)
score = evaluate(
    dataset = dataset,
    metrics=[context_recall, context_precision],
    llm=Tongyi(model_name="qwen-plus"))
score.to_pandas()

Evaluating: 100%|██████████| 6/6 [00:08<00:00,  1.36s/it]


,question,answer,ground_truth,contexts,context_recall,context_precision
0,张伟是哪个部门的？,根据提供的信息，没有提到张伟所在的部门。如果您能提供更多关于张伟的信息，我可能能够帮助您找到答案。,张伟是教研部的成员,"[提供⾏政管理与协调⽀持，优化⾏政⼯作流程。 , 绩效管理部 韩杉 李⻜ I902 041 ...",0.0,0.0
1,张伟是哪个部门的？,张伟是人事部门的,张伟是教研部的成员,"[李凯 教研部主任 , 牛顿发现了万有引力]",0.0,0.0
2,张伟是哪个部门的？,张伟是教研部的,张伟是教研部的成员,"[牛顿发现了万有引力, 张伟 教研部工程师，他最近在负责课程研发]",1.0,0.5


由上面的数据可以看到：
- 最后一行数据的回答是准确的
- 过程中检索到的参考资料（contexts）中也包含了正确答案的观点，即「张伟是教研部的」。这一情况体现在了 context recall 得分为 1。
- 但是 contexts 中并不是每一条都是和问题及答案相关的，比如「牛顿发现了万有引力」。这一情况体现在了 context precision 得分为 0.5。

#### 2.2.2 了解 context recall 和 context precision 的计算过程

##### Context recall
Context recall 的计算过程是：

1. 由大模型将 ground_truth 分解成 n 个观点（statements）。  

   比如，可以由ground_truth「张伟是教研部的成员」生成观点列表 [张伟是教研部的"]。
2. 由大模型判断每个观点能在检索到的参考资料（contexts）中找到依据，或者说 context 是否能支撑 ground_truth 的观点。  

   比如，这个观点在第三行数据的 contexts 中能找到依据「*张伟 教研部工程师，他最近在负责课程研发*」。
3. 然后 ground_truth 观点列表中，能在 contexts 中找到依据的观点占比，作为 context_recall 分数。  

   这里的得分为 1 = 1/1。

##### Context precision

Context precision 的计算过程稍微复杂一些：

1. 按顺序读取 contexts 中的 context<sub>i</sub> ，根据 question 与 ground_truth，判断 context<sub>i</sub>  是否相关。相关为 1 分，否则为 0 分。  
   
   比如上面第三行数据中，context<sub>1</sub>(牛顿发现了万有引力) 是不相关的，context<sub>2</sub> 相关。
2. 对于每一个 context，以该 context 及之前 context 的分数之和作为分子，context 所处排位作为分母，计算 precision 分。  

   对于上面第三行数据，context<sub>1</sub> precision 分为 0/1 = 0，context<sub>2</sub> precision 分为 1/2 = 0.5。
3. 对每一个 context 的 precision 分求和，除以相关的 context 个数，得到 context_precision。  

   对于上面第三行数据，context_precision = (0 + 0.5) / 1 = 0.5。

### 2.3 其他推荐了解的指标

Ragas 还提供了很多其他的指标，这里就不一一介绍，你可以访问 Ragas 的文档来查看更多指标的适用场景和工作原理。

Ragas 支持的指标：https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/

## ✅ 本节小结

通过本节内容的学习，你已经学会了怎么为RAG应用建立自动化测试了。

自动化测试是工程优化的重要手段。借助量化的自动化测试，可以帮你在改进 RAG 应用时，从**感觉**变好了，转变为指标**量化**显示应用表现更好。这不仅可以帮助你更快地评估RAG应用的问答质量、找到优化方向，还能将你所作出的优化结果量化出来。

当然，有了自动化测试并不意味着你就完全不需要人工评估了，建议在实际应用时，邀请 RAG 应用对应的领域专家一起构建能反映真实场景问题分布的测试集，并且持续更新测试集。

同时，由于大模型并不能总是做到 100% 准确，也建议你在实际使用时，定期抽样评估自动化测试结果的精度。对于 Ragas，你可以通过调整默认评测方法中的提示词（比如补充和你的业务领域相关的参考样例），来改善其表现。

## 🔥 课后小测验


【单选题】Context Recall指标检测的是？（ ）

A. 整体回答质量的评估

B. 评估与提问相关的召回文本段是否排名靠前

C. 生成的答案与召回文本段是否相关

D. 生成的答案是否与提问相关

答案：B

<br>

## ✉️ 评价反馈
感谢你学习阿里云大模型ACP认证课程，如果你觉得课程有哪里写得好、哪里写得不好，期待你[通过问卷提交评价和反馈](https://survey.aliyun.com/apps/zhiliao/Mo5O9vuie)。

你的批评和鼓励都是我们前进的动力。